# Porównanie klasyfikatora konwolucyjnego i rekurencyjnego dla NLP

W tym notebooku porównamy działanie (architekturę, złożoność, efektywność uczenia) dwóch klasyfikatorów sentymentu recenzji

#### Załadujmy zależności

In [ ]:
import tensorflow
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt

#### Ustawmy hiperparametry dla klasyfikatora konwolucyjnego

In [ ]:
# lokalizacja wyjściowa dla wag:
conv_output_dir = 'model_output/conv'

# parametry treningu:
conv_epochs = 4
conv_batch_size = 128

# osadzenie przestrzeni wektorowej: 
conv_n_dim = 64
conv_n_unique_words = 5000 
conv_max_review_length = 400
conv_pad_type = trunc_type = 'pre'

#parametr dropoutu
conv_drop_embed = 0.2

# parametry architektury konwolucyjnej:
n_conv = 256 # filtry/kernele
k_conv = 3 # długość kernela

# parametry warstwy gęstej: 
n_dense = 256
dropout = 0.2

#### Ustawmy hiperparametry dla klasyfikatora rekurencyjnego

In [ ]:
# lokalizacja wyjściowa dla wag:
rnn_output_dir = 'model_output/rnn'

# parametry treningu:
rnn_epochs = 16
rnn_batch_size = 128

# osadzenie przestrzeni wektorowej: 
rnn_n_dim = 64 
rnn_n_unique_words = 10000 
rnn_max_review_length = 100 #zmniejszone z powodu szybciej zanikającego gradientu w jednostkach rekurencyjnych
rnn_pad_type = trunc_type = 'pre'
rnn_drop_embed = 0.2 

# parametry architektury rekurencyjnej:
n_rnn = 256 
drop_rnn = 0.2

#### Załaduj dane treningowe i walidacyjne z parametrem liczby uwzględnianych słów (odrębnie dla obu klasyfikatorów)

#### Przetwórz wszystkie załadowane dane poprzez ujednolicenie wielkości danych wejściowych

In [ ]:
#dla rnn

In [ ]:
# dla conv

#### Zaprojektuj obie architektury sieci, wybierając odpowiednie zaimportowane obiekty i zadeklarowane wyżej hiperparametry

#### ConvNN
- odpowiednio osadź przestrzeń wektorową (embedding), 
- dodaj przestrzenny dropout z odpowiednim parametrem i wybraną funkcją aktywacji
- utwórz warstwę konwolucyjną jednowymiarową z odpowiednimi hiperparametrami
- dodaj warstwę redukującą - globalny jednowymiarowy max-pooling
- uzupełnij sieć o warstwę gęstą z wybraną funkcją aktywacji i dropoutem
- zakończ warstwą klasyfikującą

In [ ]:
model_conv = Sequential()
# ...

In [ ]:
model_conv.summary() 

#### RNN
- odpowiednio osadź przestrzeń wektorową (embedding), 
- dodaj przestrzenny dropout z odpowiednim parametrem i wybraną funkcją aktywacji
- utwórz prostą warstwę rekurencyjną z odpowiednimi hiperparametrami
- zakończ warstwą klasyfikującą

In [ ]:
model_rnn = Sequential()
# ...

In [ ]:
model_rnn.summary()

#### Skompiluj oba modele z odpowiednimi parametrami

#### Stwórz obiekty do rejestrowania wag dla każdego z modeli

In [ ]:
# dla rnn

In [ ]:
# dla conv

#### Naucz oba modele z użyciem zbiorów walidacyjnych, z odpowiednimi zadeklarowanymi wyżej parametrami

In [ ]:
# rnn

In [ ]:
# conv

#### Wykonaj inferencję (ewaluację) obu modeli uprzednio ładując wagi z ich najlepszych epok

#### Zestaw na jednym wykresie histogramy dla danych walidacyjnych obu modeli oraz, na kolejnym wykresie, krzywe ROC obu klasyfikatorów,  a następnie - patrząc na oba te wykresy oraz statystyki uczenia, zapisz trzy sensowne wnioski co do wyników

**tu wnioski**